In [1]:
#!pip3 install selenium bs4
#!pip3 install --upgrade requests

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import json
import os
import argparse
import cv2

import requests
import urllib
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import datetime
import time

urllib3.disable_warnings(InsecureRequestWarning)

maxcount = 10000

chromedriver = r'PATH TO YOUR DRIVER\ chromedriver.exe'

In [9]:
def get_image_url(browser):
    page_source = browser.page_source
    tattoo = []

    soup = BeautifulSoup(page_source, 'lxml')
    images = soup.find_all('img')


    for image in images:
        try:
            url = image['data-src']
            if not url.find('https://'):
                tattoo.append(url)
        except:
            try:
                url = image['src']
                if not url.find('https://'):
                    tattoo.append(image['src'])
            except Exception as e:
                print(f'No found image sources.')
                print(e)
    
    return tattoo


def download_staticimages(searchword1, website = "tattoodo"):
    urls = []
    
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    #options.add_argument('--headless')

    try:
        browser = webdriver.Chrome(chromedriver, options=options)
    except Exception as e:
        print(f'No found chromedriver in this environment.')
        print(f'Install on your machine. exception: {e}')
        sys.exit()

    browser.set_window_size(1280, 1024)
    
    if website == "tattoodo":
        
        browser.get('https://www.tattoodo.com/tattoos?q=' + searchword1)
        time.sleep(1)

        print(f'Getting you a lot of images. This may take a few moments...')

        element = browser.find_element_by_tag_name('body')


        try:
            for i in range(100):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)
                urls.extend(get_image_url(browser))
        except:
            print("Error Encountered")


        count = 0
        print("There are " + str(len(urls)) + " urls before dupes removed")
        urls = list(set(urls))
        print("There are " + str(len(urls)) + " urls after dupes removed")
        if urls:
            for url in urls:
                try:
                    res = requests.get(url, verify=False, stream=True)
                    rawdata = res.raw.read()
                    path = os.path.join(dirs, "tattoodo" + searchword1 + 'img_' + str(count) + '.jpg')
                    with open(path, 'wb') as f:
                        f.write(rawdata)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        h, w = img.shape
                        if(not (h > 40  and w > 40)):
                            os.remove(path)
                            print( str(path) + " removed")
                        else:
                            count += 1
                except Exception as e:
                    print('Failed to write rawdata.')
                    print(e)
    
    #Now for google. The primary difference here is that google requires us to click a button to show more results
    elif website == "google":
        
        browser.get('https://www.google.com/search?q=' + searchword1 + '&source=lnms&tbm=isch')
        time.sleep(1)

        print(f'Getting you a lot of images. This may take a few moments...')

        element = browser.find_element_by_tag_name('body')

        # Scroll down
        #for i in range(30):
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

        try:
            browser.find_element_by_id('smb').click()
            for i in range(50):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)
        except:
            for i in range(10):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)

        print(f'Reached end of page.')
        time.sleep(0.5)
        print(f'Retry')
        time.sleep(0.5)

        # Below is for english. Change this wording to your lanaguage if you require.
        try:
            browser.find_element_by_xpath('//input[@value="Show more results"]').click()
        except:
            print("Only one page of images")

        # Scroll down
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

        try:
            browser.find_element_by_id('smb').click()
            for i in range(50):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)
        except:
            for i in range(10):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)

        page_source = browser.page_source 

        soup = BeautifulSoup(page_source, 'lxml')
        images = soup.find_all('img')

        urls = []
        for image in images:
            try:
                url = image['data-src']
                if not url.find('https://'):
                    urls.append(url)
            except:
                try:
                    url = image['src']
                    if not url.find('https://'):
                        urls.append(image['src'])
                except Exception as e:
                    print(f'No found image sources.')
                    print(e)

        count = 0
        if urls:
            for url in urls:
                try:
                    res = requests.get(url, verify=False, stream=True)
                    rawdata = res.raw.read()
                    with open(os.path.join(dirs, searchword1 + 'img_' + str(count) + '.jpg'), 'wb') as f:
                        f.write(rawdata)
                        count += 1
                except Exception as e:
                    print('Failed to write rawdata.')
                    print(e)

    browser.close()
    return count

In [10]:
website = "google"

for j in ['scorpion', 'orca', 'skull']:
        searchterms = [j, j + ' tattoo sketch', j + ' tattoo design', j + ' tattoo drawing', j + ' tattoo flash', j + ' tattoo outline', 
                   j + ' tattoo line art', j + ' aesthetic line art tattoos', j + ' pencil art tattoo drawing', 
                   j + ' tattoo artwork',j + ' tattoo art', j + ' illustration tattoo', j + ' tattoo', j + ' idea',
                   j + ' tattoo sketch drawing ideas', 'black and white '+ j + ' drawing', 
                   j + ' sketch', j + ' drawing']

    # Main block

    dirs = j 

    for i in searchterms:
        t0 = time.time()
        maxcount = 10000

        if not os.path.exists(dirs):
            os.mkdir(dirs)

        count = download_staticimages(i, website)
        t1 = time.time()


        total_time = t1 - t0
        print(f'\n')
        print(i)
        print(f'Download completed. [Successful count = {count}].')
        print(f'Total time is {str(total_time)} seconds.')

<ipython-input-9-4f5c2444cd28>:35: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chromedriver, options=options)


Getting you a lot of images. This may take a few moments...


<ipython-input-9-4f5c2444cd28>:47: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = browser.find_element_by_tag_name('body')


There are 1674 urls before dupes removed
There are 144 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'


scorpion sketch
Download completed. [Successful count = 142].
Total time is 194.6995279788971 seconds.
Getting you a lot of images. This may take a few moments...
There are 1723 urls before dupes removed
There are 541 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'


scorpion drawing
Download completed. [Successful count = 538].
Total time is 362.7631118297577 seconds.
Getting you a lot of images. This may take a few moments...
There are 1679 urls before dupes removed
There are 518 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawda



orca tattoo drawing
Download completed. [Successful count = 434].
Total time is 215.69238591194153 seconds.
Getting you a lot of images. This may take a few moments...
There are 1738 urls before dupes removed
There are 565 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Fa

Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'


orca aesthetic line art tattoos
Download completed. [Successful count = 500].
Total time is 318.64224314689636 seconds.
Getting you a lot of images. This may take a few moments...
There are 1746 urls before dupes removed
There are 571 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribut

Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'


orca tattoo
Download completed. [Successful count = 485].
Total time is 239.55197310447693 seconds.
Getting you a lot of images. This may take a few moments...
There are 1632 urls before dupes removed
There are 494 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to 

Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'


orca sketch
Download completed. [Successful count = 481].
Total time is 219.5920968055725 seconds.
Getting you a lot of images. This may take a few moments...
There are 1675 urls before dupes removed
There are 511 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to w

Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'


skull aesthetic line art tattoos
Download completed. [Successful count = 441].
Total time is 207.87412905693054 seconds.
Getting you a lot of images. This may take a few moments...
There are 4056 urls before dupes removed
There are 520 urls after dupes removed
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribute 'shape'
Failed to write rawdata.
'NoneType' object has no attribu



skull drawing
Download completed. [Successful count = 445].
Total time is 195.99454426765442 seconds.
